## Solving Ill Conditioned System of Linear Equations

In [1]:
import numpy as np

# Define a nearly singular matrix
A = np.array([[1, 1],
              [1, 1.0001]], dtype=float)

# Define right-hand side
b = np.array([2, 2.0001], dtype=float)

# Exact solution using numpy solver
x_exact = np.linalg.solve(A, b)

Solution using [Cramer's rule](https://en.wikipedia.org/wiki/Cramer%27s_rule)

In [2]:
def cramer_rule(A, b):
    det = A[0,0]*A[1,1] - A[0,1]*A[1,0]
    x = np.array([(b[0]*A[1,1] - b[1]*A[0,1]) / det,
                  (A[0,0]*b[1] - A[1,0]*b[0]) / det])
    return x

x_cramer = cramer_rule(A, b)

Perturb b slightly

In [3]:
b_perturbed = b + np.array([1e-5, -1e-5])
x_perturbed = np.linalg.solve(A, b_perturbed)

Print results

In [4]:
print("Matrix A:")
print(A)
print("\nOriginal b:", b)
print("Solution (numpy):", x_exact)
print("Solution (Cramer):", x_cramer)
print("\nPerturbed b:", b_perturbed)
print("Solution for perturbed b:", x_perturbed)

def forward_error(x_true, x_computed):
    return np.linalg.norm(x_true - x_computed) / np.linalg.norm(x_true)

def backward_error(A, b, x):
    return np.linalg.norm(b - A @ x)/(np.linalg.norm(b) + np.linalg.norm(A,2)*np.linalg.norm(x))

print("\nRelative error in solution due to small perturbation in b:", forward_error(x_exact, x_perturbed))

Matrix A:
[[1.     1.    ]
 [1.     1.0001]]

Original b: [2.     2.0001]
Solution (numpy): [1. 1.]
Solution (Cramer): [1. 1.]

Perturbed b: [2.00001 2.00009]
Solution for perturbed b: [1.20001 0.8    ]

Relative error in solution due to small perturbation in b: 0.20000500006383065


In [5]:
# TODO compute the relative backward error.
# Is np.linalg.solve() backward stable?
print("Relative backward error:", backward_error(A, b_perturbed, x_perturbed))

Relative backward error: 0.0


In [6]:
# Compute the solution of perturbed system using Cramer's rule. Check the relative error and backward stability.
# Is Cramer's rule backward stable?
x_perturbed = cramer_rule(A, b_perturbed)
x_exact = np.array([1.20001, 0.8]) # by hand
print("\nRelative error in solution (Cramer):", forward_error(x_exact, x_perturbed))
print("Relative backward error (Cramer):", backward_error(A, b_perturbed, x_perturbed))


Relative error in solution (Cramer): 6.172805172534171e-13
Relative backward error (Cramer): 2.0480084815858047e-13


That is about 3 orders of magnitude above the machine precision. Is it stable? No, see below.

In [7]:
A = np.array([[1, 1],
              [1, 1 + 1e-15]], dtype=float)
x_exact = np.array([1, 1 + 1-15], dtype=float)
b = A @ x_exact
x = cramer_rule(A, b)
print("\nRelative error in solution (Cramer):", forward_error(x_exact, x))
print("Relative backward error(Cramer):", backward_error(A, b, x))


Relative error in solution (Cramer): 0.07821538888750428
Relative backward error(Cramer): 0.02657636584658371
